In [1]:
!pip install hdbscan
!pip install sentence_transformers
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039283 sha256=eaf7d7ec421627698c9f2844eb7866308308e88635aa9dc1c3944a0d941fbe45
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
Successfully built hdbscan
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.8
    Uninstalling Cython-3.0.8:
      Successfully uninstalled Cython-3.0.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import tensorflow as tf

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel,pipeline

import hdbscan
from sklearn import metrics


from sentence_transformers import SentenceTransformer
import umap

In [19]:
model_id = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)
classifier = pipeline('text-classification', tokenizer=tokenizer, model=model, return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
The model 'BertModel' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', '

In [20]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [21]:
aggregated_data = train.groupby("category")["text"].apply(lambda x: " ".join(x)).reset_index()

aggregated_data

,category,text
0,Refund_not_showing_up,I don't see my refund money yet in my account....
1,activate_my_card,Please help me with my card. It won't activat...
2,age_limit,I want to open an account for my children How ...
3,apple_pay_or_google_pay,"Hi, I have an apple watch. How do I use it to ..."
4,atm_support,I can use this card at which ATMs? Can I use t...
...,...,...
72,virtual_card_not_working,Why isn't my disposable virtual card working? ...
73,visa_or_mastercard,Is there a way I can have a mastercard? is it ...
74,why_verify_identity,Why do you have an identity check? I do not fe...
75,wrong_amount_of_cash_received,Why did I only receive a partial amount of wha...


In [22]:
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        normed_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
    return embeddings

In [23]:
#aggregated_data['bert_embeddings'] = aggregated_data['text'].apply(lambda x: get_bert_embeddings(x))

#print(aggregated_data.head())

In [24]:
#aggregated_data.head()

In [25]:
bert_embeddings = aggregated_data['text'].apply(lambda x: get_bert_embeddings(x))
print(bert_embeddings)

0     [[tensor(-0.2145), tensor(-0.0850), tensor(0.2...
1     [[tensor(0.0172), tensor(-0.1680), tensor(0.29...
2     [[tensor(0.0514), tensor(0.0432), tensor(0.214...
3     [[tensor(-0.1866), tensor(-0.0686), tensor(0.2...
4     [[tensor(-0.1690), tensor(-0.0749), tensor(0.3...
                            ...                        
72    [[tensor(0.0966), tensor(-0.2345), tensor(0.38...
73    [[tensor(0.1161), tensor(-0.1861), tensor(0.32...
74    [[tensor(0.0791), tensor(0.0919), tensor(0.136...
75    [[tensor(-0.3698), tensor(-0.1893), tensor(0.1...
76    [[tensor(-0.3159), tensor(-0.1084), tensor(0.2...
Name: text, Length: 77, dtype: object


In [30]:
print(bert_embeddings.shape)

(77,)


In [31]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000,
                                   min_df=1,
                                   max_df=0.85,
                                   ngram_range=(1, 2))

In [32]:
X_tfidf = tfidf_vectorizer.fit_transform(aggregated_data['text'])

In [33]:
X_tfidf.shape

(77, 1000)

In [34]:
print(X_tfidf.toarray())

[[0.         0.         0.00902898 ... 0.01262894 0.00707061 0.        ]
 [0.         0.         0.00819875 ... 0.00254837 0.00570707 0.        ]
 [0.         0.         0.01975013 ... 0.         0.0412436  0.        ]
 ...
 [0.         0.         0.00722214 ... 0.         0.         0.        ]
 [0.15760424 0.17925056 0.00435062 ... 0.         0.00908528 0.        ]
 [0.         0.         0.         ... 0.         0.01352128 0.00480681]]


How to compare the effect of embeddings and tfidf feature matrix? Compare the models (Bert and others using tfidf methods?)

In [16]:
tfidf_matrix=X_tfidf.toarray()

In [52]:
"""
embeddings_matrix = np.array(bert_embeddings)

bert_embeddings_np = bert_embeddings.to_numpy()
bert_embeddings_np = np.stack(bert_embeddings_np)
bert_embeddings_np_2 = bert_embeddings_np.reshape(bert_embeddings_np.shape[0], bert_embeddings_np.shape[2])


pca = PCA(n_components=2)
tfidf_pca = pca.fit_transform(tfidf_matrix)
embeddings_pca = pca.fit_transform(bert_embeddings_np_2)


plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(tfidf_pca[:, 0], tfidf_pca[:, 1], label='TF-IDF PCA')
plt.title("Tfidf PCA")
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(embeddings_pca[:, 0], embeddings_pca[:, 1], label='Embeddings PCA')
plt.title("Embeddings PCA")
plt.legend()

plt.show()
"""

'\nembeddings_matrix = np.array(bert_embeddings)\n\nbert_embeddings_np = bert_embeddings.to_numpy()\nbert_embeddings_np = np.stack(bert_embeddings_np)\nbert_embeddings_np_2 = bert_embeddings_np.reshape(bert_embeddings_np.shape[0], bert_embeddings_np.shape[2])\n\n\npca = PCA(n_components=2)\ntfidf_pca = pca.fit_transform(tfidf_matrix)\nembeddings_pca = pca.fit_transform(bert_embeddings_np_2)\n\n\nplt.figure(figsize=(12, 6))\n\nplt.subplot(1, 2, 1)\nplt.scatter(tfidf_pca[:, 0], tfidf_pca[:, 1], label=\'TF-IDF PCA\')\nplt.title("Tfidf PCA")\nplt.legend()\n\nplt.subplot(1, 2, 2)\nplt.scatter(embeddings_pca[:, 0], embeddings_pca[:, 1], label=\'Embeddings PCA\')\nplt.title("Embeddings PCA")\nplt.legend()\n\nplt.show()\n'